# Message Placeholders in LangChain Tutorial

This comprehensive tutorial covers how to use placeholders in different types of messages when working with LLMs using LangChain and OpenAI.

## What You'll Learn

- **ChatPromptTemplate**: Building structured prompts with variables
- **Message Types**: SystemMessage, HumanMessage, AIMessage with placeholders
- **MessagesPlaceholder**: Dynamic conversation history insertion
- **Practical Applications**: Real-world use cases and patterns

## Prerequisites

- Python 3.8+
- OpenAI API key
- Basic understanding of LLMs and LangChain

## 1. Setup and Installation

First, let's install the required packages and set up our environment.

In [ ]:
# Install required packages (uncomment if needed)
# !pip install langchain-openai langchain-core python-dotenv

In [ ]:
# Import required libraries
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

# Load environment variables
load_dotenv()

# Verify API key is loaded
if not os.getenv("OPENAI_API_KEY"):
    raise ValueError("OPENAI_API_KEY not found in environment variables")

# Initialize the model
model = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.7)

print("✅ Environment setup complete!")

## 2. ChatPromptTemplate Basics

### What are Placeholders?

Placeholders are variables in your prompt templates that get replaced with actual values at runtime. They are denoted by curly braces `{variable_name}`.

`ChatPromptTemplate` is the primary way to create structured prompts with placeholders in LangChain.

### Example 1: Simple Single-Message Template

In [ ]:
# Simple template with one variable
chat_template = ChatPromptTemplate.from_template(
    "Tell me a joke about {topic}."
)

# Format and invoke
messages = chat_template.format_messages(topic="programming")

print("Formatted Messages:")
for msg in messages:
    print(f"{type(msg).__name__}: {msg.content}")

print("\n" + "="*60 + "\n")

response = model.invoke(messages)
print("Model Response:")
print(response.content)

### Example 2: Template with Multiple Variables

In [ ]:
# Template with multiple variables
chat_template = ChatPromptTemplate.from_template(
    "Translate the following {source_lang} text to {target_lang}: {text}"
)

messages = chat_template.format_messages(
    source_lang="English",
    target_lang="French",
    text="Hello, how are you?"
)

print("Formatted Messages:")
for msg in messages:
    print(f"{type(msg).__name__}: {msg.content}")

print("\n" + "="*60 + "\n")

response = model.invoke(messages)
print("Model Response:")
print(response.content)

## 3. ChatPromptTemplate with Multiple Message Types

The real power of `ChatPromptTemplate` comes from `from_messages()` which allows you to create structured conversations with different message types.

### Message Type Tuples

You can specify messages using tuples: `("role", "content with {placeholders}")`

Supported roles:
- `"system"` - System messages (set AI behavior)
- `"human"` or `"user"` - User messages
- `"ai"` or `"assistant"` - AI assistant messages

### Example 3: System + Human Messages

In [ ]:
# Create a template with system and human messages
chat_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a {role} who specializes in {specialty}. Your tone should be {tone}."),
    ("human", "{user_input}")
])

# Format with values
messages = chat_prompt.format_messages(
    role="Python expert",
    specialty="data science",
    tone="friendly and educational",
    user_input="Explain what pandas is."
)

print("Formatted Messages:")
for msg in messages:
    print(f"\n{type(msg).__name__}:")
    print(msg.content)

print("\n" + "="*60 + "\n")

response = model.invoke(messages)
print("Model Response:")
print(response.content)

### Example 4: Multi-Turn Conversation Template

In [ ]:
# Create a multi-turn conversation template
chat_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a {role}. Your expertise is in {domain}."),
    ("human", "I have a question about {topic}."),
    ("ai", "I'd be happy to help you with {topic}. What specifically would you like to know?"),
    ("human", "{question}")
])

messages = chat_prompt.format_messages(
    role="senior software engineer",
    domain="cloud architecture",
    topic="AWS Lambda",
    question="What are the best practices for Lambda cold starts?"
)

print("Formatted Conversation:")
for i, msg in enumerate(messages, 1):
    print(f"\n{i}. {type(msg).__name__}:")
    print(msg.content)

print("\n" + "="*60 + "\n")

response = model.invoke(messages)
print("Model Response:")
print(response.content)

### Example 5: Complex System Message with Structured Content

In [ ]:
# Template with detailed, structured system message
chat_prompt = ChatPromptTemplate.from_messages([
    ("system", """You are a {role}.

Your responsibilities:
- {responsibility_1}
- {responsibility_2}
- {responsibility_3}

Communication style: {style}
Target audience: {audience}
"""),
    ("human", "{task}")
])

messages = chat_prompt.format_messages(
    role="technical documentation writer",
    responsibility_1="Explain complex concepts clearly",
    responsibility_2="Provide code examples",
    responsibility_3="Include best practices",
    style="concise and practical",
    audience="beginner developers",
    task="Explain what a REST API is."
)

print("Formatted Messages:")
for msg in messages:
    print(f"\n{type(msg).__name__}:")
    print(msg.content)

print("\n" + "="*60 + "\n")

response = model.invoke(messages)
print("Model Response:")
print(response.content)

### Example 6: Few-Shot Learning Pattern

In [ ]:
# Few-shot learning with examples
chat_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a {task_type} assistant. Follow the pattern shown in the examples."),
    ("human", "{example_input_1}"),
    ("ai", "{example_output_1}"),
    ("human", "{example_input_2}"),
    ("ai", "{example_output_2}"),
    ("human", "{actual_input}")
])

messages = chat_prompt.format_messages(
    task_type="sentiment analysis",
    example_input_1="I love this product!",
    example_output_1="Positive",
    example_input_2="This is terrible.",
    example_output_2="Negative",
    actual_input="It's okay, nothing special."
)

print("Few-Shot Learning Template:")
for i, msg in enumerate(messages, 1):
    print(f"\n{i}. {type(msg).__name__}: {msg.content}")

print("\n" + "="*60 + "\n")

response = model.invoke(messages)
print("Model Response:")
print(response.content)

## 4. MessagesPlaceholder - Dynamic Conversation History

`MessagesPlaceholder` is a special placeholder that allows you to insert a list of messages dynamically. This is crucial for maintaining conversation history.

### Example 7: Basic MessagesPlaceholder Usage

In [ ]:
# Create a prompt with a placeholder for conversation history
chat_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful AI assistant. Remember the conversation context."),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{user_input}")
])

# Simulate a conversation history
conversation_history = [
    HumanMessage(content="My name is Alice."),
    AIMessage(content="Nice to meet you, Alice! How can I help you today?"),
    HumanMessage(content="I'm learning Python."),
    AIMessage(content="That's great! Python is a wonderful language to learn. What aspect of Python are you focusing on?")
]

# Format with history and new input
messages = chat_prompt.format_messages(
    chat_history=conversation_history,
    user_input="What was my name again?"
)

print("Full Conversation:")
for i, msg in enumerate(messages, 1):
    print(f"\n{i}. {type(msg).__name__}:")
    print(msg.content)

print("\n" + "="*60 + "\n")

response = model.invoke(messages)
print("Model Response:")
print(response.content)

### Example 8: Interactive Conversation Loop

In [ ]:
# Build a conversational loop
chat_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a friendly chatbot. Keep track of the conversation."),
    MessagesPlaceholder(variable_name="history"),
    ("human", "{input}")
])

# Initialize conversation history
history = []

# Simulate a multi-turn conversation
user_inputs = [
    "Hi! I'm planning a trip to Japan.",
    "What are the must-visit places?",
    "How about food recommendations?"
]

for user_input in user_inputs:
    print(f"\n{'='*60}")
    print(f"User: {user_input}")
    print(f"{'='*60}\n")
    
    # Format messages with current history
    messages = chat_prompt.format_messages(
        history=history,
        input=user_input
    )
    
    # Get response
    response = model.invoke(messages)
    print(f"Assistant: {response.content}\n")
    
    # Update history
    history.append(HumanMessage(content=user_input))
    history.append(AIMessage(content=response.content))

print("\n" + "="*60)
print(f"Total messages in history: {len(history)}")

### Example 9: Optional MessagesPlaceholder

In [ ]:
# Optional history - works with or without chat_history
chat_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant."),
    MessagesPlaceholder(variable_name="chat_history", optional=True),
    ("human", "{question}")
])

# First call - without history
print("First call (no history):")
print("="*60)
messages = chat_prompt.format_messages(
    question="What is machine learning?"
)
response1 = model.invoke(messages)
print(response1.content)

# Second call - with history
print("\n" + "="*60)
print("Second call (with history):")
print("="*60)
messages = chat_prompt.format_messages(
    chat_history=[
        HumanMessage(content="What is machine learning?"),
        AIMessage(content=response1.content)
    ],
    question="Can you give me an example?"
)
response2 = model.invoke(messages)
print(response2.content)

## 5. Practical Use Cases

Let's explore real-world applications using ChatPromptTemplate.

### Use Case 1: Customer Support Chatbot

In [ ]:
# Customer support template
support_prompt = ChatPromptTemplate.from_messages([
    ("system", """You are a customer support agent for {company_name}.
Product: {product_name}
Customer tier: {customer_tier}

Guidelines:
- Be professional and empathetic
- Provide clear solutions
- Escalate if needed
"""),
    MessagesPlaceholder(variable_name="conversation_history"),
    ("human", "Customer issue: {issue}")
])

# Simulate a support interaction
messages = support_prompt.format_messages(
    company_name="CloudHost Solutions",
    product_name="Premium Hosting Plan",
    customer_tier="Gold",
    conversation_history=[
        HumanMessage(content="My website is down!"),
        AIMessage(content="I'm sorry to hear that. Let me check your server status immediately.")
    ],
    issue="The website has been down for 30 minutes now."
)

print("Customer Support Conversation:")
for msg in messages:
    print(f"\n{type(msg).__name__}:")
    print(msg.content)

print("\n" + "="*60 + "\n")

response = model.invoke(messages)
print("Support Agent Response:")
print(response.content)

### Use Case 2: Code Review Assistant

In [ ]:
# Code review template
code_review_prompt = ChatPromptTemplate.from_messages([
    ("system", """You are an expert code reviewer specializing in {language}.
Focus areas: {focus_areas}
Review style: {review_style}

Provide constructive feedback on:
1. Code quality
2. Best practices
3. Potential bugs
4. Performance improvements
"""),
    ("human", """Please review this code:

```{language}
{code}
```

Specific concerns: {concerns}
""")
])

code_sample = """def calculate_total(items):
    total = 0
    for item in items:
        total = total + item['price'] * item['quantity']
    return total
"""

messages = code_review_prompt.format_messages(
    language="Python",
    focus_areas="performance, readability, error handling",
    review_style="detailed and educational",
    code=code_sample,
    concerns="Is this function efficient? Should I add error handling?"
)

print("Code Review Request:")
for msg in messages:
    print(f"\n{type(msg).__name__}:")
    print(msg.content[:200] + "..." if len(msg.content) > 200 else msg.content)

print("\n" + "="*60 + "\n")

response = model.invoke(messages)
print("Code Review:")
print(response.content)

### Use Case 3: Multi-Language Translation Service

In [ ]:
# Translation template with context
translation_prompt = ChatPromptTemplate.from_messages([
    ("system", """You are a professional translator.
Source language: {source_lang}
Target language: {target_lang}
Context: {context}
Tone: {tone}

Provide accurate, culturally appropriate translations.
"""),
    ("human", "Translate: {text}")
])

# Example translations
translations = [
    {
        "source_lang": "English",
        "target_lang": "Spanish",
        "context": "business email",
        "tone": "formal",
        "text": "We appreciate your continued partnership."
    },
    {
        "source_lang": "English",
        "target_lang": "Japanese",
        "context": "casual conversation",
        "tone": "friendly",
        "text": "Let's grab coffee sometime!"
    }
]

for i, trans in enumerate(translations, 1):
    print(f"\n{'='*60}")
    print(f"Translation {i}")
    print(f"{'='*60}\n")
    
    messages = translation_prompt.format_messages(**trans)
    response = model.invoke(messages)
    
    print(f"Original ({trans['source_lang']}): {trans['text']}")
    print(f"Translation ({trans['target_lang']}): {response.content}")
    print(f"Context: {trans['context']}, Tone: {trans['tone']}")

### Use Case 4: RAG (Retrieval Augmented Generation) Pattern

In [ ]:
# RAG template with context injection
rag_prompt = ChatPromptTemplate.from_messages([
    ("system", """You are a helpful assistant that answers questions based on provided context.

Instructions:
- Use ONLY the information from the context below
- If the answer is not in the context, say so
- Cite specific parts of the context when answering

Context:
{context}
"""),
    MessagesPlaceholder(variable_name="chat_history", optional=True),
    ("human", "{question}")
])

# Simulate retrieved context
context = """LangChain is a framework for developing applications powered by language models.
It provides several key features:
1. Prompt templates for dynamic prompt construction
2. Chains for combining multiple LLM calls
3. Agents for decision-making and tool use
4. Memory for maintaining conversation state
5. Integration with various LLM providers like OpenAI, Anthropic, and more.
"""

# First question
messages = rag_prompt.format_messages(
    context=context,
    question="What is LangChain?"
)

print("Question 1: What is LangChain?")
print("="*60)
response1 = model.invoke(messages)
print(response1.content)

# Follow-up question with history
print("\n" + "="*60 + "\n")
print("Question 2: What are its key features?")
print("="*60)

messages = rag_prompt.format_messages(
    context=context,
    chat_history=[
        HumanMessage(content="What is LangChain?"),
        AIMessage(content=response1.content)
    ],
    question="What are its key features?"
)

response2 = model.invoke(messages)
print(response2.content)

### Use Case 5: Content Generation with Style Guide

In [ ]:
# Content generation with detailed style parameters
content_prompt = ChatPromptTemplate.from_messages([
    ("system", """You are a {content_type} writer.

Style Guide:
- Tone: {tone}
- Target audience: {audience}
- Length: {length}
- Key themes: {themes}

Writing rules:
- {rule_1}
- {rule_2}
- {rule_3}
"""),
    ("human", "Topic: {topic}\n\nAdditional instructions: {instructions}")
])

messages = content_prompt.format_messages(
    content_type="blog post",
    tone="professional yet approachable",
    audience="tech-savvy professionals",
    length="medium (300-400 words)",
    themes="innovation, practical applications, future trends",
    rule_1="Use concrete examples",
    rule_2="Avoid jargon unless explained",
    rule_3="Include actionable takeaways",
    topic="The Impact of AI on Software Development",
    instructions="Focus on how AI tools are changing daily workflows for developers."
)

print("Content Generation Request:")
print("="*60)
for msg in messages:
    print(f"\n{type(msg).__name__}:")
    print(msg.content[:150] + "..." if len(msg.content) > 150 else msg.content)

print("\n" + "="*60 + "\n")

response = model.invoke(messages)
print("Generated Content:")
print(response.content)

### Use Case 6: Interview Preparation Assistant

In [ ]:
# Interview preparation chatbot
interview_prompt = ChatPromptTemplate.from_messages([
    ("system", """You are an interview preparation coach.

Interview Details:
- Company: {company}
- Position: {position}
- Interview type: {interview_type}
- Candidate background: {background}

Your role:
- Ask relevant interview questions
- Provide constructive feedback
- Suggest improvements
"""),
    MessagesPlaceholder(variable_name="interview_history"),
    ("human", "{candidate_response}")
])

# Simulate interview interaction
messages = interview_prompt.format_messages(
    company="TechCorp",
    position="Senior Python Developer",
    interview_type="technical",
    background="5 years experience in backend development",
    interview_history=[
        AIMessage(content="Let's start with a technical question: Can you explain the difference between a list and a tuple in Python?"),
    ],
    candidate_response="Lists are mutable and tuples are immutable. Lists use square brackets and tuples use parentheses."
)

print("Interview Simulation:")
print("="*60)
for msg in messages:
    print(f"\n{type(msg).__name__}:")
    print(msg.content)

print("\n" + "="*60 + "\n")

response = model.invoke(messages)
print("Interviewer Response:")
print(response.content)

## 6. Advanced ChatPromptTemplate Patterns

### Pattern 1: Dynamic Template Building

In [ ]:
# Build templates dynamically based on conditions
def create_assistant_prompt(include_examples=True, include_history=True):
    messages = [
        ("system", "You are a {role} assistant.")
    ]
    
    if include_examples:
        messages.extend([
            ("human", "{example_question}"),
            ("ai", "{example_answer}")
        ])
    
    if include_history:
        messages.append(MessagesPlaceholder(variable_name="history", optional=True))
    
    messages.append(("human", "{question}"))
    
    return ChatPromptTemplate.from_messages(messages)

# Create template with examples and history
prompt = create_assistant_prompt(include_examples=True, include_history=True)

messages = prompt.format_messages(
    role="Python tutor",
    example_question="What is a decorator?",
    example_answer="A decorator is a function that modifies the behavior of another function.",
    question="Can you show me an example of a decorator?"
)

print("Dynamic Template Example:")
print("="*60)
for i, msg in enumerate(messages, 1):
    print(f"\n{i}. {type(msg).__name__}: {msg.content}")

print("\n" + "="*60 + "\n")

response = model.invoke(messages)
print("Response:")
print(response.content)

### Pattern 2: Template Chaining

In [ ]:
# Chain multiple prompts together
# Step 1: Generate ideas
idea_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a creative brainstorming assistant."),
    ("human", "Generate 3 {content_type} ideas about {topic}. List them briefly.")
])

ideas_messages = idea_prompt.format_messages(
    content_type="blog post",
    topic="sustainable living"
)

print("Step 1: Generating Ideas")
print("="*60)
ideas_response = model.invoke(ideas_messages)
print(ideas_response.content)

# Step 2: Expand on the best idea
expand_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a content writer."),
    ("human", """Here are some ideas:
{ideas}

Pick the most interesting one and write a detailed outline for it.""")
])

expand_messages = expand_prompt.format_messages(
    ideas=ideas_response.content
)

print("\n" + "="*60)
print("Step 2: Expanding Best Idea")
print("="*60)
expand_response = model.invoke(expand_messages)
print(expand_response.content)

### Pattern 3: Reusable Template Factory

In [ ]:
# Create a reusable template factory class
class PromptFactory:
    """Factory for creating common prompt templates."""
    
    @staticmethod
    def create_qa_prompt(domain: str, style: str = "concise"):
        """Create a Q&A prompt for a specific domain."""
        return ChatPromptTemplate.from_messages([
            ("system", f"You are a {domain} expert. Answer questions in a {style} style."),
            MessagesPlaceholder(variable_name="history", optional=True),
            ("human", "{question}")
        ])
    
    @staticmethod
    def create_task_prompt(task_type: str, requirements: list):
        """Create a task-oriented prompt."""
        req_text = "\n".join([f"- {req}" for req in requirements])
        return ChatPromptTemplate.from_messages([
            ("system", f"""You are a {task_type} assistant.

Requirements:
{req_text}
"""),
            ("human", "{task}")
        ])

# Usage
qa_prompt = PromptFactory.create_qa_prompt("machine learning", "detailed")
messages = qa_prompt.format_messages(
    question="What is overfitting?"
)

print("Reusable Template Factory Example:")
print("="*60)
response = model.invoke(messages)
print(response.content)

## 7. Best Practices

### Key Recommendations

1. **Use `ChatPromptTemplate.from_messages()`**: This is the recommended way to create prompts
2. **Clear Variable Names**: Use descriptive names like `user_question` instead of `q`
3. **Consistent Formatting**: Maintain consistent placeholder style across templates
4. **Template Reusability**: Create reusable templates for common patterns
5. **Optional Placeholders**: Use `optional=True` in MessagesPlaceholder when appropriate
6. **Structured System Messages**: Organize system messages with clear sections
7. **Message Type Tuples**: Use simple tuples `("role", "content")` for clarity

In [ ]:
# Best practices example
import pandas as pd

# Summary of common patterns
patterns = {
    'Pattern': [
        'Simple Template',
        'System + Human',
        'Multi-Turn',
        'Few-Shot Learning',
        'With History',
        'Optional History',
        'RAG Pattern',
        'Dynamic Building'
    ],
    'Use Case': [
        'Single query/response',
        'Setting AI behavior',
        'Simulating conversation',
        'Learning from examples',
        'Stateful conversations',
        'Flexible history handling',
        'Context-based Q&A',
        'Conditional templates'
    ],
    'Key Component': [
        'from_template()',
        '("system", ...), ("human", ...)',
        'Multiple message tuples',
        'Example human/ai pairs',
        'MessagesPlaceholder',
        'optional=True',
        'Context in system message',
        'Dynamic message list'
    ]
}

df = pd.DataFrame(patterns)
print("\nChatPromptTemplate Patterns Summary:")
print("="*100)
print(df.to_string(index=False))
print("="*100)

## 8. Conclusion

### Key Takeaways

1. **ChatPromptTemplate** is the primary tool for creating structured prompts
2. **from_messages()** allows flexible multi-message templates
3. **Message tuples** `("role", "content")` are simple and powerful
4. **MessagesPlaceholder** is essential for conversation history
5. **Optional placeholders** provide flexibility
6. **Template reusability** improves code maintainability

### Next Steps

- Experiment with different message combinations
- Build reusable template libraries for your use cases
- Integrate with LangChain chains and agents
- Explore prompt versioning and management
- Implement conversation memory systems

### Resources

- [LangChain Prompts Documentation](https://python.langchain.com/docs/modules/model_io/prompts/)
- [ChatPromptTemplate API Reference](https://api.python.langchain.com/en/latest/prompts/langchain_core.prompts.chat.ChatPromptTemplate.html)
- [LangChain Expression Language (LCEL)](https://python.langchain.com/docs/expression_language/)

## Practice Exercises

Try these exercises to reinforce your understanding:

1. **Exercise 1**: Create a job interview chatbot template with company, position, and interviewer style placeholders
2. **Exercise 2**: Build a multi-turn conversation system with MessagesPlaceholder for a travel planning assistant
3. **Exercise 3**: Create a few-shot learning template for sentiment analysis
4. **Exercise 4**: Implement a RAG pattern template for a technical documentation Q&A system
5. **Exercise 5**: Design a code generation template with language, framework, and requirements placeholders

In [ ]:
# Exercise workspace - try your own templates here!

# Example: Your custom template
# my_prompt = ChatPromptTemplate.from_messages([
#     ("system", "..."),
#     ("human", "...")
# ])

# Test your template
# messages = my_prompt.format_messages(...)
# response = model.invoke(messages)
# print(response.content)